In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv', index_col=0)

In [ ]:
df.head()

In [ ]:
# Selección de columnas numéricas
numerical_columns = ['Sleep Duration', 'Physical Activity Level', 'Stress Level', 'Heart Rate', 'Daily Steps']

# Calcular media
mean_values = df[numerical_columns].mean()

# Calcular mediana
median_values = df[numerical_columns].median()

# Calcular moda
mode_values = df[numerical_columns].mode().iloc[0]  # mode() devuelve un DataFrame

# Calcular desviación estándar
std_dev_values = df[numerical_columns].std()

# Calcular rango
range_values = df[numerical_columns].max() - df[numerical_columns].min()

# Calcular percentiles (25%, 50%, 75%)
percentiles = df[numerical_columns].quantile([0.25, 0.5, 0.75])

# Mostrar resultados
print("Media:\n", mean_values)
print("\nMediana:\n", median_values)
print("\nModa:\n", mode_values)
print("\nDesviación Estándar:\n", std_dev_values)
print("\nRango:\n", range_values)
print("\nPercentiles (25%, 50%, 75%):\n", percentiles)


### Valores atípicos

In [ ]:
# Calcular IQR para identificar valores atípicos
Q1 = df[numerical_columns].quantile(0.25)
Q3 = df[numerical_columns].quantile(0.75)
IQR = Q3 - Q1

# Identificar valores atípicos
outliers = ((df[numerical_columns] < (Q1 - 1.5 * IQR)) | (df[numerical_columns] > (Q3 + 1.5 * IQR)))

# Mostrar los valores atípicos
print("Outliers detectados:\n", df[outliers.any(axis=1)])


### Resumen

In [ ]:
summary_table = pd.DataFrame({
    'Media': mean_values,
    'Mediana': median_values,
    'Moda': mode_values,
    'Desviación Estándar': std_dev_values,
    'Rango': range_values
})

# Agregar los percentiles a la tabla
summary_table['25%'] = percentiles.loc[0.25].values
summary_table['50%'] = percentiles.loc[0.5].values
summary_table['75%'] = percentiles.loc[0.75].values

print("Tabla Resumen:\n", summary_table)


#### Hstogramas para cada columna numérica

In [ ]:
df[numerical_columns].hist(bins=15, figsize=(15, 10), layout=(2, 3), color='skyblue', edgecolor='black')
plt.suptitle('Histogramas de Variables Numéricas')
plt.show()


#### Boxplot para identificar los datos atípicos

In [ ]:
# Crear boxplots para cada columna numérica
plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_columns, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=col, data=df, color='orange')
    plt.title(f'Boxplot de {col}')

plt.tight_layout()
plt.show()


#### Gráfico de Barras para variables categóricas

In [ ]:
# Gráfico de barras para la variable 'Occupation'
plt.figure(figsize=(10, 6))
sns.countplot(x='Occupation', data=df, hue='Occupation', palette='viridis')
plt.title('Distribución de Occupation')
plt.xticks(rotation=45)
plt.show()

# Gráfico de barras para la variable 'Sleep Disorder'
plt.figure(figsize=(10, 6))
sns.countplot(x='Sleep Disorder', data=df, hue='Sleep Disorder', palette='viridis')
plt.title('Distribución de Sleep Disorder')
plt.xticks(rotation=45)
plt.show()



#### Relaciones entre variables

In [ ]:
# Relación entre Sleep Duration y Stress Level
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Sleep Duration', y='Stress Level', hue='BMI Category', style='Gender', data=df)
plt.title('Relación entre Sleep Duration y Stress Level')
plt.show()

# Relación entre Physical Activity Level y Sleep Duration segmentado por Sleep Disorder
plt.figure(figsize=(10, 6))
sns.boxplot(x='Sleep Disorder', y='Physical Activity Level', data=df, hue='Sleep Disorder', palette='Set2')
plt.title('Actividad Física vs Sleep Disorder')
plt.show()


#### Gráfica de Correlaciones

In [ ]:
# Calcular la matriz de correlación
correlation_matrix = df[numerical_columns].corr()

# Graficar el mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de Calor de la Matriz de Correlación')
plt.show()


#### Graficar la distribución de las variables categóricas con violinplots

In [ ]:
# Violinplots para explorar variables numéricas segmentadas por variables categóricas
plt.figure(figsize=(15, 10))

# Violinplot para 'Sleep Duration' segmentado por 'BMI Category'
plt.subplot(2, 2, 1)
sns.violinplot(x='BMI Category', y='Sleep Duration', data=df, hue='BMI Category', palette='muted')
plt.title('Sleep Duration por BMI Category')

# Violinplot para 'Physical Activity Level' segmentado por 'BMI Category'
plt.subplot(2, 2, 2)
sns.violinplot(x='BMI Category', y='Physical Activity Level', data=df, hue='BMI Category', palette='muted')
plt.title('Physical Activity Level por BMI Category')

# Violinplot para 'Heart Rate' segmentado por 'Gender'
plt.subplot(2, 2, 3)
sns.violinplot(x='Gender', y='Heart Rate', data=df, hue='Gender', palette='muted')
plt.title('Heart Rate por Gender')

# Violinplot para 'Stress Level' segmentado por 'Sleep Disorder'
plt.subplot(2, 2, 4)
sns.violinplot(x='Sleep Disorder', y='Stress Level', data=df, hue='Sleep Disorder', palette='muted')
plt.title('Stress Level por Sleep Disorder')

plt.tight_layout()
plt.show()


#### Modelo de regresión lineal simple

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Seleccionar las características y la variable objetivo
X = df[['Stress Level']]
y = df['Sleep Duration']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

# Graficar la línea de regresión
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test['Stress Level'], y=y_test, label='Actual', color='blue')
sns.lineplot(x=X_test['Stress Level'], y=y_pred, label='Predicted', color='red')
plt.title('Regresión Lineal: Sleep Duration vs Stress Level')
plt.show()


#### Relación entre variables categoricas y numéricas

In [ ]:
# Boxplot para explorar la relación entre 'Occupation' y 'Sleep Duration'
plt.figure(figsize=(10, 6))
sns.boxplot(x='Occupation', y='Sleep Duration', data=df, hue='Occupation', palette='Set3')
plt.title('Sleep Duration por Occupation')
plt.xticks(rotation=45)
plt.show()

# Boxplot para explorar la relación entre 'Sleep Disorder' y 'Heart Rate'
plt.figure(figsize=(10, 6))
sns.boxplot(x='Sleep Disorder', y='Heart Rate', data=df, hue='Sleep Disorder', palette='Set3')
plt.title('Heart Rate por Sleep Disorder')
plt.show()


#### Visualizar con Pairplot

In [ ]:
sns.pairplot(df, hue='Sleep Disorder', vars=numerical_columns, palette='husl')
plt.suptitle('Relaciones entre variables numéricas y Sleep Disorder', y=1.02)
plt.show()
